# Compare Models

Create object to automated model comparisons

In [1]:
import arithmetic_datasets as ad

from tensorflow.keras.layers import Input, Lambda, Dense, concatenate
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt
import random
import numpy as np

from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import tensorflow as tf
import random
import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Metric

Using TensorFlow backend.


### ModelComparator

```
mc = ModelComparator()
mc.add_model(name, model)
mc.add_training_set(name, data)

mc.train_models(epochs, batches, **kwargs)

mc.compare_min(metric)
mc.compare_max(metric)

mc.history_for(model_name, training_set_name)
```

In [2]:
class ModelComparator:
    def __init__(self):
        self.models = {}
        self.training_sets = {}
        self.histories = {}
    
    def add_model(self, name, model):
        self.models[name] = model
    
    def add_training_set(self, name, data):
        self.training_sets[name] = data
        
    def train_models(self, **kwargs):
        for model_name in self.models:
            for tset_name in self.training_sets:
                print(f"MODEL {model_name} TRAINING ON DATASET {tset_name}")
                
                model = self.models[model_name]()
                
                history = model.fit(
                    self.training_sets[tset_name][0], self.training_sets[tset_name][1],
                    **kwargs
                )
                
                self.histories[(model_name, tset_name)] = history
    
    def compare_min(self, metric):
        result = {}
        minimum = None
        min_name = None
        for hkey in self.histories:
            history = self.histories[hkey].history
            result[hkey] = np.amin(history[metric])
            if minimum == None or minimum >= result[hkey]:
                minimum = result[hkey]
                min_name = hkey
        return result, {"name" : min_name, "value":minimum}
            
    
    def compare_max(self, metric):
        result = {}
        maximum = None
        max_name = None
        for hkey in self.histories:
            history = self.histories[hkey].history
            result[hkey] = np.amax(history[metric])
            if maximum == None or maximum >= result[hkey]:
                maximum = result[hkey]
                max_name = hkey
        return result, {"name" : max_name, "value": maximum}
    
    def history_for(self, model_name, training_set_name):
        return self.histories[(model_name, training_set_name)]

### Create Models

In [3]:
def nondense_model():
    # Input layer of 3 neurons 
    inp = Input(shape=(1,3))
    
    #128 layer
    d2_out = Dense(128)(inp)

    #grab first, 2nd half of the 128 layer
    d2_out_p1 = Lambda(lambda x: x[:,:,0:64])(d2_out)
    d2_out_p2 = Lambda(lambda x: x[:,:,64:128])(d2_out)

    #64 layer(s)
    d3_out = Dense(64)(d2_out_p1)
    d4_out = Dense(64)(d2_out_p2)

    #grab output nodes from both 64 layers
    d5_out = concatenate([d3_out, d4_out])
    
    o = Dense(1)(d5_out)
    
    model = Model(inp, o)

    opt = keras.optimizers.Adam(learning_rate=0.01)
    model.compile(
        loss="MeanSquaredError",
        metrics = ["accuracy"],
        optimizer = opt
    )
    
    
    return model

def dense_model_5L():
    model_5layer = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1024, input_shape=(1,3)),
        tf.keras.layers.Dense(512),
        tf.keras.layers.Dense(256),
        tf.keras.layers.Dense(128),
        tf.keras.layers.Dense(64),
        tf.keras.layers.Dense(1)
    ])
    
    opt = keras.optimizers.Adam(learning_rate=0.01)
    model.compile(
        loss="MeanSquaredError",
        metrics = ["accuracy"],
        optimizer = opt
    )
    

    return model_5layer

def dense_model2():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(2048, input_shape=(1,3)))
    
    model.add(tf.keras.layers.Dense(1024))
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Dense(64))
    model.add(tf.keras.layers.Dense(1))
    
    opt = keras.optimizers.Adam(learning_rate=0.01)
    model.compile(
        loss="MeanSquaredError",
        metrics = ["accuracy"],
        optimizer = opt
    )
    
    return model

### Datasets

In [4]:
num_examples = 20000
rstart = 1
rend = 10000

# MULT
mult_setX, mult_setY = ad.gen_data_mult(num_examples, rstart, rend)
mult_setX = mult_setX.reshape(num_examples, 1, 3)

# LOG(MULT)
mult_logX = np.log(mult_setX)
mult_logY = np.log(mult_setY)

### Build ModelComparator

In [5]:
mc = ModelComparator()

mc.add_model("nondense_model", nondense_model)
mc.add_model("dense_model", dense_model_5L)
mc.add_model("dense_model v2", dense_model2)

mc.add_training_set("log normal multiply", (mult_logX, mult_logY))
mc.add_training_set("multiplication", (mult_setX, mult_setY))

### Train Models

In [6]:
num_epochs = 1000
batch_size = num_examples

mc.train_models(
    batch_size=batch_size,
    epochs=num_epochs
)

MODEL nondense_model TRAINING ON DATASET log normal multiply


ValueError: ('Could not interpret optimizer identifier:', <keras.optimizers.Adam object at 0x000002107A774460>)

### compare min loss

In [ ]:
models_min_loss, min_loss = mc.compare_min("loss")

In [ ]:
models_min_loss